In [1]:
# ebablbe auto-completion
%config Completer.use_jedi = False

In [1]:
import csv
import ipyparams
import matplotlib.pyplot as plt
import nltk
import numpy as np
import os
import pandas as pd
import random
import re
import shutil
import string  
import sys
import tensorflow as tf
import tensorflow_hub as hub

from collections import Counter
# from google_trans_new import google_translator
from joblib import Parallel, delayed
from nltk.corpus import stopwords
from scipy import stats
from sklearn.feature_extraction import text
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from tensorflow.keras import Model, models, layers, regularizers, preprocessing, datasets, metrics, losses, optimizers
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorboard.plugins.hparams import api as hp 

base_path = os.path.abspath('/tf/eipi10/jian-xu3/snps-classification')
sys.path.append(base_path)

import snps 

from snps.core import text_predictor 
from snps.core import util 
from snps.utils import data
from snps.core.text_classification import lr_schedule
from snps.core.text_classification import plot_distribution, plot_length_distribution, plot_frequency_distribution
from snps.core.text_classification import Params, TextClassificationHelper
from snps.core.text_classification import SimpleTextDatasets, SequenceTextDatasets, RawTextDatasets,TransferTextDatasets

<IPython.core.display.Javascript object>

2021-07-05 08:45:26,327: INFO: deployment = dev
2021-07-05 08:45:26,329: INFO: dataset_name = snps_18510_c8
2021-07-05 08:45:26,330: INFO: working_path = /tf/eipi10/jian-xu3/snps-classification


In [2]:
#!/usr/bin/env python3

import json
import os
import os.path
import posixpath
import subprocess
import pandas as pd
import psutil
import requests
from IPython.display import display


def show_notebooks_table(host, port):
    """Show table with info about running jupyter notebooks.

    Args:
        host: host of the jupyter server.
        port: port of the jupyter server.

    Returns:
        DataFrame with rows corresponding to running notebooks and following columns:
            * index: notebook kernel id.
            * path: path to notebook file.
            * pid: pid of the notebook process.
            * memory: notebook memory consumption in percentage.
    """
    pd.set_option('display.max_colwidth', 300)
    
    notebooks = get_running_notebooks(host, port)
    prefix = long_substr([notebook['path'] for notebook in notebooks])
    df = pd.DataFrame(notebooks)
    df = df.set_index('kernel_id')
    df.index.name = prefix
    df.path = df.path.apply(lambda x: x[len(prefix):])
    df['pid'] = df.apply(lambda row: get_process_id(row.name), axis=1)
    # same notebook can be run in multiple processes
    df = expand_column(df, 'pid')
    df['memory'] = df.pid.apply(memory_usage_psutil)
    df['cpu'] = df.pid.apply(cpu_usage_psutil)
    df['num_threads'] = df.pid.apply(num_threads_psutil)    
    df = df.sort_values('memory', ascending=False)
    df.index = range(1, len(df)+1)
    return df


def get_running_notebooks(host, port, password='xxw'):
    """
    Get kernel ids and paths of the running notebooks.

    Args:
        host: host at which the notebook server is listening. E.g. 'localhost'.
        port: port at which the notebook server is listening. E.g. 8888.
            
    Returns:
        list of dicts {kernel_id: notebook kernel id, path: path to notebook file}.
    """
        
    BASE_URL = 'http://{0}:{1}/'.format(host, port)
        
    # Get the cookie data
    s = requests.Session()
    url = BASE_URL + 'login?next=%2F'
    resp = s.get(url)
    xsrf_cookie = resp.cookies['_xsrf']

    # Login with the password
    params = {'_xsrf': xsrf_cookie, 'password': password}
    res = s.post(url, data=params)

    # Find which kernel corresponds to which notebook
    # by querying the notebook server api for sessions
    url = posixpath.join(BASE_URL, 'api', 'sessions')
    ret = s.get(url)
    #print('Status code:', ret.status_code)

    # Get the notebook list
    res = json.loads(ret.text)
    notebooks = [{'kernel_id': notebook['kernel']['id'],
                  'path': notebook['notebook']['path']} for notebook in res]
    return notebooks


def get_process_id(name):
    """Return process ids found by (partial) name or regex.

    Source: https://stackoverflow.com/a/44712205/304209.
    >>> get_process_id('kthreadd')
    [2]
    >>> get_process_id('watchdog')
    [10, 11, 16, 21, 26, 31, 36, 41, 46, 51, 56, 61]  # ymmv
    >>> get_process_id('non-existent process')
    []
    """
    child = subprocess.Popen(['pgrep', '-f', name], stdout=subprocess.PIPE, shell=False)
    response = child.communicate()[0]
    return [int(pid) for pid in response.split()]


def memory_usage_psutil(pid=None):
    """Get memory usage percentage by current process or by process specified by id, like in top.

    Source: https://stackoverflow.com/a/30014612/304209.

    Args:
        pid: pid of the process to analyze. If None, analyze the current process.

    Returns:
        memory usage of the process, in percentage like in top, values in [0, 100].
    """
    if pid is None:
        pid = os.getpid()
    process = psutil.Process(pid)
    return round(process.memory_percent(),3)

def cpu_usage_psutil(pid=None):
    """Get cpu usage percentage by current process or by process specified by id, like in top.

    Source: https://stackoverflow.com/a/30014612/304209.

    Args:
        pid: pid of the process to analyze. If None, analyze the current process.

    Returns:
        cpu usage of the process, in percentage like in top, values in [0, 100].
    """
    if pid is None:
        pid = os.getpid()
    process = psutil.Process(pid)
    return round(process.cpu_percent(),3)


def num_threads_psutil(pid=None):
    """Get num_threads by current process or by process specified by id, like in top.

    Source: https://stackoverflow.com/a/30014612/304209.

    Args:
        pid: pid of the process to analyze. If None, analyze the current process.

    Returns:
        num_threads of the process
    """
    if pid is None:
        pid = os.getpid()
    process = psutil.Process(pid)
    return process.num_threads()


def long_substr(strings):
    """Find longest common substring in a list of strings.

    Source: https://stackoverflow.com/a/2894073/304209.

    Args:
        strings: list of strings.

    Returns:
        longest substring which is found in all of the strings.
    """
    substr = ''
    if len(strings) > 1 and len(strings[0]) > 0:
        for i in range(len(strings[0])):
            for j in range(len(strings[0])-i+1):
                if j > len(substr) and all(strings[0][i:i+j] in x for x in strings):
                    substr = strings[0][i:i+j]
    return substr


def expand_column(dataframe, column):
    """Transform iterable column values into multiple rows.

    Source: https://stackoverflow.com/a/27266225/304209.

    Args:
        dataframe: DataFrame to process.
        column: name of the column to expand.

    Returns:
        copy of the DataFrame with the following updates:
            * for rows where column contains only 1 value, keep them as is.
            * for rows where column contains a list of values, transform them
                into multiple rows, each of which contains one value from the list in column.
    """
    tmp_df = dataframe.apply(
        lambda row: pd.Series(row[column]), axis=1).stack().reset_index(level=1, drop=True)
    tmp_df.name = column
    return dataframe.drop(column, axis=1).join(tmp_df)

df = show_notebooks_table('15.15.166.35', 18888)
display(df)

,path,pid,memory,cpu,num_threads
1,eipi10/xuxiangwen.github.io/_notes/00.ipynb,47,0.377,0.0,8


In [3]:
import ast
import types
import astunparse

func_def = \
"""
def add(x, y):
    return x + y
print(add(3, 5))
"""

cm = compile(func_def, '<string>', 'exec')
assert(isinstance(cm, types.CodeType))

exec(cm)

8


In [4]:
# 生成AST
r_node = ast.parse(func_def)
# 也可调用ast.dump(r_node)，但它不能进行缩进输出
ast_output = astunparse.dump(r_node)  
print(ast_output)

# 从AST得到源代码
print('-'*50)
print(astunparse.unparse(r_node))

Module(body=[
  FunctionDef(
    name='add',
    args=arguments(
      args=[
        arg(
          arg='x',
          annotation=None),
        arg(
          arg='y',
          annotation=None)],
      vararg=None,
      kwonlyargs=[],
      kw_defaults=[],
      kwarg=None,
      defaults=[]),
    body=[Return(value=BinOp(
      left=Name(
        id='x',
        ctx=Load()),
      op=Add(),
      right=Name(
        id='y',
        ctx=Load())))],
    decorator_list=[],
    returns=None),
  Expr(value=Call(
    func=Name(
      id='print',
      ctx=Load()),
    args=[Call(
      func=Name(
        id='add',
        ctx=Load()),
      args=[
        Num(n=3),
        Num(n=5)],
      keywords=[])],
    keywords=[]))])
--------------------------------------------------


def add(x, y):
    return (x + y)
print(add(3, 5))



In [5]:
class CodeVisitor(ast.NodeVisitor):
    def visit_BinOp(self, node):
        if isinstance(node.op, ast.Add):
            node.op = ast.Sub()
        self.generic_visit(node)

    def visit_FunctionDef(self, node):
        print('Function Name:%s'% node.name)
        self.generic_visit(node)
        func_log_stmt = ast.print(
            dest = None,
            values = [ast.Str(s = 'calling func: %s' % node.name, lineno = 0, col_offset = 0)],
            nl = True,
            lineno = 0,
            col_offset = 0,
        )
        node.body.insert(0, func_log_stmt)

r_node = ast.parse(func_def)
visitor = CodeVisitor()
visitor.visit(r_node)

print(astunparse.unparse(r_node))
exec(compile(r_node, '<string>', 'exec'))

Function Name:add


AttributeError: module 'ast' has no attribute 'print'

In [ ]:
!pip install -U ast


In [ ]:
dir(ast)

In [ ]:
import collections, re
pairs = collections.defaultdict(int)
pairs['\u2581 3', '4']=5
pairs['4', '5']=3
pairs

In [ ]:
def merge_vocab(pair):
    v_out = {}
    bigram = re.escape(' '.join(pair))
    p = re.compile(r'(?<!\S)' + bigram + r'(?!\S)')

    w_out = p.sub(''.join(pair), word)
    return w_out

bigrams = [re.escape(' '.join(pair)) for pair in pairs]
print(f'bigrams={bigrams}')
ps = [re.compile(r'(?<!\S)' + bigram + r'(?!\S)') for bigram in bigrams] 
print(f'ps={ps}')

In [ ]:
pair = list(pairs.keys())[0]
bigram = re.escape(' '.join(pair))
p = re.compile(r'(?<!\S)' + bigram + r'(?!\S)')
print(p)
word=r'3\\ 4'
p.sub(''.join(pair), word)

In [ ]:
''.join(pair)

In [ ]:
word='3\\ 4'
word 

In [ ]:
word='3\\ 4'
word

In [ ]:
def sub(pair, word):
    '''在word中替换pair为pair去空格'''
    print(f'word = {word}')
    bigram = re.escape(' '.join(pair))

    # 匹配，前后都为空的pair
    # (?<!\S)	匹配前面不是非空的位置，即匹配前面是空
    # (?!\S)	匹配后面跟的不是非空的位置，即匹配后面是空      
    pattern = re.compile(r'(?<!\S)' + bigram + r'(?!\S)')
    print(f'pattern = {pattern}')
    new_word = pattern.sub(''.join(pair), word)
    print(f'new_word = {new_word}') 

word = 'b c ab c b c c bd b c'

pair = ('b', 'c')
# 从结果可以看到，第一个,第三个，第五个b c被替换成了bc
sub(pair, word)

In [ ]:

# word = re.escape('\u2581abcd') 
# print(word)


In [ ]:
import re

pattern = '\S'
string = ' 344-Hello--World!'
result = re.search(pattern, string)
print(result)
print(result.group())

In [ ]:
pattern = '[a-z]+'
string = '-----2344-HELLO--WORLD!'
result = re.search(pattern, string)
print(result)

In [ ]:
pair = ('4', '5')
bigram = re.escape(' '.join(pair))
p = re.compile(bigram)
print(p)
word = re.escape('1234 567') 
print(word)
print(''.join(pair))
print(p.sub(''.join(pair), word))

In [ ]:
import numpy as np

x1 = np.array([0.2, 0.3, 0.1, 0.4])
scores = np.array([2, 3, 4, -1])
order = scores.argsort()[::-1] 
print(order)
i = order[0]
np.maximum(x1[i], x1[order[1:]])

In [ ]:
def py_cpu_nms(dets, thresh):
    """
    dets是shape为 Nx5 的ndarray,如:
    dets = np.array([[5,5,20,20,0.7],
                     [0,0,10,10,0.3]])
    
    """
    x1 = dets[:, 0] #所有的x1
    y1 = dets[:, 1] #所有的y1
    x2 = dets[:, 2] #所有的x2
    y2 = dets[:, 3] #所有的y2
    scores = dets[:, 4] #所有的得分

    areas = (x2 - x1 + 1) * (y2 - y1 + 1) #所有框的面积
    order = scores.argsort()[::-1] #面积排序

    keep = [] #
    while order.size > 0:
        i = order[0]
        keep.append(i)
        xx1 = np.maximum(x1[i], x1[order[1:]])
        yy1 = np.maximum(y1[i], y1[order[1:]])
        xx2 = np.minimum(x2[i], x2[order[1:]])
        yy2 = np.minimum(y2[i], y2[order[1:]])
        
        print(x1[order[1:]])
        print(xx1,yy1,xx2,yy2)

        w = np.maximum(0.0, xx2 - xx1 + 1)
        h = np.maximum(0.0, yy2 - yy1 + 1)
        inter = w * h

        ovr = inter / (areas[i] + areas[order[1:]] - inter)

        inds = np.where(ovr <= thresh)[0]
        order = order[inds + 1]

    return keep

In [ ]:
import numpy as np


In [ ]:
a =1


In [ ]:
!pip install  jupyter_contrib_nbextensions
!pip install jupyter_nbextensions_configurator

In [ ]:
!jupyter contrib nbextension install --user 
!jupyter nbextensions_configurator enable --user

In [ ]:
1

In [ ]:
a=1

In [ ]:
import numpy as np

In [ ]:
np.testing.

In [6]:
561*728*5

2042040

In [7]:

class Dog:
 
    def bark(self):
        print("汪汪叫")
 
# 子类  继承
class XiaoTianQuan(Dog):
 
    def fly(self):
        print("我会飞")
 
    # 可以重写父类中的同名方法
    def bark(self):
 
        # super().父类方法名 调用父类中的方法 (第一种方式)(推荐)
        super().bark()
 
        # 父类名.方法(self) 调用父类中的方法 (第二种方式,python2.x)(不推荐,父类名修改后,此处也得改)
        Dog.bark(self)
 
        # 注意：如果使用子类名调用方法，可能会出现递归调用 -- 死循环！
        # XiaoTianQuan.bark(self)  # 会产生死循环
 
        # 针对子类特有的需求，进行扩展
        print("神一样的叫唤...")
 
 
xtq = XiaoTianQuan()
xtq.bark()


汪汪叫
汪汪叫
神一样的叫唤...


In [14]:
a = lambda x, y: x+y, x-y

NameError: name 'x' is not defined

In [13]:
a(2, 3)

(5, -1)

In [16]:
r= np.array([[1, 2, 3], [3, 4, 2], [0, 0, 0]])

In [18]:
r[np.all(r == 0, axis=1)]

array([[0, 0, 0]])

In [19]:
r[~np.all(r == 0, axis=1)]

array([[1, 2, 3],
       [3, 4, 2]])

In [2]:
!pwd

/tf/eipi10/xuxiangwen.github.io/_notes


In [9]:
import os
dataset_path='/tf/eipi10/xuxiangwen.github.io/_notes/'


'_notes'

In [17]:
def f2(a, b=2, c=3):
    return a + b + c

In [29]:
a = ['1', '2', '3']
a[1:2]

['2']

In [ ]:
class TextReader:
    """Print and number lines in a text file."""

    def __init__(self, filename):
        self.filename = filename
        self.file = open(filename)
        self.lineno = 0

    def readline(self):
        self.lineno += 1
        line = self.file.readline()
        if not line:
            return None
        if line.endswith('\n'):
            line = line[:-1]
        return "%i: %s" % (self.lineno, line)

    def __getstate__(self):
        # Copy the object's state from self.__dict__ which contains
        # all our instance attributes. Always use the dict.copy()
        # method to avoid modifying the original state.
        state = self.__dict__.copy()
        # Remove the unpicklable entries.
        del state['file']
        return state

    def __setstate__(self, state):
        # Restore instance attributes (i.e., filename and lineno).
        self.__dict__.update(state)
        # Restore the previously opened file's state. To do so, we need to
        # reopen it and read from it until the line count is restored.
        file = open(self.filename)
        for _ in range(self.lineno):
            file.readline()
        # Finally, save the file.
        self.file = file
        
reader = TextReader("hello.txt")
print(reader.readline())
print(reader.readline())
s = pickle.dumps(reader)
#print(s)
new_reader = pickle.loads(s)
print(new_reader.readline())

In [30]:
!pwd

/tf/eipi10/xuxiangwen.github.io/_notes


In [31]:
import os
os.path.dirname('/tf/eipi10/xuxiangwen.github.io/_notes')

'/tf/eipi10/xuxiangwen.github.io'

In [43]:
a = {'a':1, 'b':2}
class B:
    def __init__(self):
        self.c = {'e':2, 'a':3}

dict(a, **B().c)

{'a': 3, 'b': 2, 'e': 2}

In [42]:
b.c

AttributeError: 'dict' object has no attribute 'c'

In [2]:
60000/32

1875.0

In [5]:
import datetime
now = datetime.datetime.now()
now.strftime("%Y-%m-%d %H:%M")

'2021-08-10 09:38'

In [6]:
datetime.datetime.now().strftime("%Y-%m-%d %H:%M")

'2021-08-10 09:39'

In [8]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense,Conv2D,TimeDistributed, Input
 
input_ = Input(shape=(12,8))
out = TimeDistributed(Dense(units=10))(input_)
#out = Dense(units=10)(input_)
model = Model(inputs=input_,outputs=out)
model.summary()


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 12, 8)]           0         
_________________________________________________________________
time_distributed (TimeDistri (None, 12, 10)            90        
Total params: 90
Trainable params: 90
Non-trainable params: 0
_________________________________________________________________


In [10]:
input_ = Input(shape=(12,8))
# out = TimeDistributed(Dense(units=10))(input_)
out = Dense(units=10)(input_)
model = Model(inputs=input_,outputs=out)
model.summary()


Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 12, 8)]           0         
_________________________________________________________________
dense_1 (Dense)              (None, 12, 10)            90        
Total params: 90
Trainable params: 90
Non-trainable params: 0
_________________________________________________________________


In [20]:

input_ = Input(shape=(12,32,32, 2))
out = TimeDistributed(Conv2D(filters=7,kernel_size=(3,3),padding='same'))(input_)
# out = Conv2D(filters=7,kernel_size=(3,3),padding='same')(input_)
model = Model(inputs=input_,outputs=out)
model.summary()


Model: "model_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_13 (InputLayer)        [(None, 12, 32, 32, 2)]   0         
_________________________________________________________________
time_distributed_8 (TimeDist (None, 12, 32, 32, 7)     133       
Total params: 133
Trainable params: 133
Non-trainable params: 0
_________________________________________________________________


In [21]:
input_ = Input(shape=(12,32,32, 2))
# out = TimeDistributed(Conv2D(filters=7,kernel_size=(3,3),padding='same'))(input_)
out = Conv2D(filters=7,kernel_size=(3,3),padding='same')(input_)
model = Model(inputs=input_,outputs=out)
model.summary()


Model: "model_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_14 (InputLayer)        [(None, 12, 32, 32, 2)]   0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 12, 32, 32, 7)     133       
Total params: 133
Trainable params: 133
Non-trainable params: 0
_________________________________________________________________


In [1]:
!pip install -U pip setuptools wheel
!pip install -U spacy
!python -m spacy download en_core_web_sm

     |████████████████████████████████| 1.6 MB 230 kB/s eta 0:00:01
     |████████████████████████████████| 819 kB 17.4 MB/s eta 0:00:01
  Attempting uninstall: wheel
    Found existing installation: wheel 0.36.2
    Uninstalling wheel-0.36.2:
      Successfully uninstalled wheel-0.36.2
  Attempting uninstall: setuptools
    Found existing installation: setuptools 56.2.0
    Uninstalling setuptools-56.2.0:
      Successfully uninstalled setuptools-56.2.0
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
2021-08-18 00:22:13.421465: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
     |████████████████████████████████| 13.6 MB 13.2 MB/s eta 0:00:01


✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [1]:
import spacy

spacy.prefer_gpu()

True

In [4]:
!pip install -U spacy[cuda112]

     |████████████████████████████████| 76.0 MB 13 kB/s s eta 0:00:01     |████▌                           | 10.6 MB 22.6 MB/s eta 0:00:03     |████▉                           | 11.6 MB 22.6 MB/s eta 0:00:03     |█████▊                          | 13.5 MB 22.6 MB/s eta 0:00:03     |███████████▊                    | 27.8 MB 25.4 MB/s eta 0:00:02     |█████████████                   | 30.6 MB 25.4 MB/s eta 0:00:02     |██████████████                  | 33.5 MB 2.4 MB/s eta 0:00:18     |███████████████████████▎        | 55.3 MB 14.5 MB/s eta 0:00:02     |████████████████████████▏       | 57.3 MB 14.5 MB/s eta 0:00:02     |█████████████████████████▊      | 61.1 MB 14.5 MB/s eta 0:00:02     |███████████████████████████▊    | 65.9 MB 19.1 MB/s eta 0:00:01


In [50]:
import spacy
from spacy import displacy
from spacy.tokens import Doc
from spacy.vocab import Vocab

spacy.prefer_gpu()
nlp = spacy.load("en_core_web_sm")
doc = nlp("This is a sentence.")

In [2]:
doc = nlp("Apple isn't looking at buying U.K. startup for $1 billion")
for token in doc:
    print(token.text, token.pos_, token.dep_)

Apple PROPN nsubj
is AUX aux
n't PART neg
looking VERB ROOT
at ADP prep
buying VERB pcomp
U.K. PROPN compound
startup NOUN dobj
for ADP prep
$ SYM quantmod
1 NUM compound
billion NUM pobj


In [106]:
import json
import requests
from spacy.lang.zh import Chinese


COUNTRIES = ['阿富汗', '奥兰群岛', '阿尔巴尼亚', '阿尔及利亚', '美属萨摩亚', '安道尔', 
             '安哥拉', '安圭拉', '安提瓜和巴布达', '阿根廷', '亚美尼亚', '阿鲁巴', 
             '澳大利亚', '奥地利', '阿塞拜疆', '孟加拉', '巴林', '巴哈马', '巴巴多斯', 
             '白俄罗斯', '比利时', '伯利兹', '贝宁', '百慕大', '不丹', '玻利维亚', 
             '波斯尼亚和黑塞哥维那', '博茨瓦纳', '布维岛', '巴西', '文莱', '保加利亚', 
             '布基纳法索', '布隆迪', '柬埔寨', '喀麦隆', '加拿大', '佛得角', '中非', 
             '乍得', '智利', '圣诞岛', '科科斯群岛', '哥伦比亚', '科摩罗', '刚果', 
             '刚果', '库克群岛', '哥斯达黎加', '科特迪瓦', '中国', '克罗地亚', '古巴', 
             '捷克', '塞浦路斯', '丹麦', '吉布提', '多米尼加', '东帝汶', '厄瓜多尔', '埃及', 
             '赤道几内亚', '厄立特里亚', '爱沙尼亚', '埃塞俄比亚', '法罗群岛', '斐济', '法国', 
             '法国大都会', '法属圭亚那', '法属波利尼西亚', '加蓬', '冈比亚', '格鲁吉亚', '德国', 
             '加纳', '直布罗陀', '希腊', '格林纳达', '瓜德罗普岛', '关岛', '危地马拉', '根西岛', 
             '几内亚比绍', '几内亚', '圭亚那', '海地', '洪都拉斯', '匈牙利', '冰岛', '印度', 
             '印度尼西亚', '伊朗', '伊拉克', '爱尔兰', '马恩岛', '以色列', '意大利', '牙买加', 
             '日本', '泽西岛', '约旦', '哈萨克斯坦', '肯尼亚', '基里巴斯', '韩国', '朝鲜', 
             '科威特', '吉尔吉斯斯坦', '老挝', '拉脱维亚', '黎巴嫩', '莱索托', '利比里亚', 
             '利比亚', '列支敦士登', '立陶宛', '卢森堡', '马其顿', '马拉维', '马来西亚', 
             '马达加斯加', '马尔代夫', '马里', '马耳他', '马绍尔群岛', '马提尼克岛', 
             '毛里塔尼亚', '毛里求斯', '马约特', '墨西哥', '密克罗尼西亚', '摩尔多瓦', 
             '摩纳哥', '蒙古', '黑山', '蒙特塞拉特', '摩洛哥', '莫桑比克', '缅甸', 
             '纳米比亚', '瑙鲁', '尼泊尔', '荷兰', '新喀里多尼亚', '新西兰', '尼加拉瓜', 
             '尼日尔', '尼日利亚', '纽埃', '诺福克岛', '挪威', '阿曼', '巴基斯坦', '帕劳', 
             '巴勒斯坦', '巴拿马', '巴布亚新几内亚', '巴拉圭', '秘鲁', '菲律宾', '皮特凯恩群岛', 
             '波兰', '葡萄牙', '波多黎各', '卡塔尔', '留尼汪岛', '罗马尼亚', '卢旺达', '俄罗斯联邦', 
             '圣赫勒拿', '圣基茨和尼维斯', '圣卢西亚', '圣文森特和格林纳丁斯', '萨尔瓦多', '萨摩亚', 
             '圣马力诺', '圣多美和普林西比', '沙特阿拉伯', '塞内加尔', '塞舌尔', '塞拉利昂', '新加坡', 
             '塞尔维亚', '斯洛伐克', '斯洛文尼亚', '所罗门群岛', '索马里', '南非', '西班牙', '斯里兰卡', 
             '苏丹', '苏里南', '斯威士兰', '瑞典', '瑞士', '叙利亚', '塔吉克斯坦', '坦桑尼亚', '泰国', 
             '特立尼达和多巴哥', '东帝汶', '多哥', '托克劳', '汤加', '突尼斯', '土耳其', '土库曼斯坦', 
             '图瓦卢', '乌干达', '乌克兰', '阿拉伯联合酋长国', '英国', '美国', '乌拉圭', '乌兹别克斯坦', 
             '瓦努阿图', '梵蒂冈', '委内瑞拉', '越南', '瓦利斯群岛和富图纳群岛', '西撒哈拉', '也门', 
             '南斯拉夫', '赞比亚', '津巴布韦']

TEXT = """冷战结束后，联合国在维和方面有显著的扩展，在十年内的维和行动数量超过过去四十年。在1988年和2000年间，通过的安理会决议数量翻了超过一倍，维和的预算也增加超过十倍。联合国调停结束萨尔瓦多内战，成功进行纳米比亚的维和任务，并监督了南非种族隔离和柬埔寨红色高棉统治之后的民主选举。1991年，联合国授权美国领导的同盟军事行动，击退伊拉克对科威特的入侵。1971年至1985年担任副秘书长的布莱恩·厄克特后来说这些行动的成功为联合国带来一种“虚假复兴”——在之后，联合国的任务变得更为艰难。

联合国宪章原本目的是阻止一个国家对另一个国家的进攻，但在90年代初联合国面临许多同时发生的严重国内危机，包括索马里、海地、莫桑比克和前南斯拉夫。在美国于摩加迪沙之战中损失惨重而撤出之后，联合国在索马里的行动被普遍认为是失败的；而联合国在波黑的行动则因为面对种族清洗显得任务不清和不果决而成为“世界的笑柄”。1994年，联合国卢旺达援助团在安理会无法决断的情况下没能对卢旺达大屠杀进行介入。

在冷战的最后数十年中，联合国受到来自美国和欧洲的批评家的批评，被指管理不善和腐败。1984年，美国总统罗纳德·里根停止对联合国教科文组织的资助，英国和新加坡也紧随其后。1992年至1996年在任的联合国秘书长布特罗斯·布特罗斯-加利对秘书处进行改革，在一定程度上精简了组织的规模。他的继任者科菲·安南（1997年-2006年在任）面对美国撤资的威胁进行进一步的管理改革。

20世纪90年代末和21世纪初，联合国授权的国际介入形式多样。联合国在塞拉利昂的任务得到英国皇家海军陆战队的支持，而对阿富汗的入侵则由北约领衔。2003年，在安理会决议未能通过授权的情况下，美国仍然入侵伊拉克，新一轮的对联合国是否有效的争论就此展开。在秘书长潘基文任下，联合国维和行动介入苏丹的达尔富尔冲突和刚果民主共和国的基伍冲突，并派出观察员和化学武器核查人员前往叙利亚内战。2013年，对联合国于2009年在斯里兰卡内战末期的行动的内部审查表明，该组织遭遇“系统性的失败”。在2010年海地地震中，有101名联合国人员殉职，这是联合国历史上最为惨痛的损失。"""

nlp = spacy.load("zh_core_web_sm")
matcher = PhraseMatcher(nlp.vocab)
patterns = list(nlp.pipe(COUNTRIES))
matcher.add("COUNTRY", None, *patterns)

# 创建一个doc并重置其已有的实体
doc = nlp(TEXT)
doc.ents = []

# 遍历所有的匹配结果
for match_id, start, end in matcher(doc):
    # 创建一个标签为"GPE"的span
    span = Span(doc, start, end, label='GPE')

    # 覆盖doc.ents并添加这个span
    doc.ents = list(doc.ents) + [span]

    # 获取这个span的根头词符
    span_root_head = span.root.head 

    # 打印这个span的根头词符的文本及span的文本
    print(span_root_head.text, "-->", span.text)

# 打印文档中的所有实体
print([(ent.text, ent.label_) for ent in doc.ents if ent.label_ == "GPE"])

内战 --> 萨尔瓦多
内战
任务 --> 纳米比亚
任务
隔离 --> 南非
隔离
统治 --> 柬埔寨
统治
授权 --> 美国
授权
入侵 --> 伊拉克
入侵
入侵 --> 科威特
入侵
莫桑比克 --> 索马里
莫桑比克
莫桑比克 --> 海地
莫桑比克
南斯拉夫 --> 莫桑比克
南斯拉夫
包括 --> 南斯拉夫
包括
惨重 --> 美国
惨重
行动 --> 索马里
行动
援助团 --> 卢旺达
援助团
大屠杀 --> 卢旺达
大屠杀
欧洲 --> 美国
欧洲
总统 --> 美国
总统
新加坡 --> 英国
新加坡
紧随 --> 新加坡
紧随
撤资 --> 美国
撤资
任务 --> 塞拉利昂
任务
陆战队 --> 英国
陆战队
入侵 --> 阿富汗
入侵
入侵 --> 美国
入侵
入侵 --> 伊拉克
入侵
介入 --> 苏丹
介入
共和国 --> 刚果
共和国
内战 --> 叙利亚
内战
内战 --> 斯里兰卡
内战
地震 --> 海地
地震
[('萨尔瓦多', 'GPE'), ('纳米比亚', 'GPE'), ('南非', 'GPE'), ('柬埔寨', 'GPE'), ('美国', 'GPE'), ('伊拉克', 'GPE'), ('科威特', 'GPE'), ('索马里', 'GPE'), ('海地', 'GPE'), ('莫桑比克', 'GPE'), ('南斯拉夫', 'GPE'), ('美国', 'GPE'), ('索马里', 'GPE'), ('卢旺达', 'GPE'), ('卢旺达', 'GPE'), ('美国', 'GPE'), ('美国', 'GPE'), ('英国', 'GPE'), ('新加坡', 'GPE'), ('美国', 'GPE'), ('塞拉利昂', 'GPE'), ('英国', 'GPE'), ('阿富汗', 'GPE'), ('美国', 'GPE'), ('伊拉克', 'GPE'), ('苏丹', 'GPE'), ('刚果', 'GPE'), ('叙利亚', 'GPE'), ('斯里兰卡', 'GPE'), ('海地', 'GPE')]


In [54]:
import spacy
from spacy.matcher import Matcher

nlp = spacy.load("en_core_web_sm")
matcher = Matcher(nlp.vocab)
# Add match ID "HelloWorld" with no callback and one pattern
patterns = [
    [{"LOWER": "hello"}, {"IS_PUNCT": True}, {"LOWER": "world"}],
    [{"LOWER": "hello"}, {"LOWER": "world"}]
]
matcher.add("HelloWorld", patterns)

doc = nlp("Hello, world! Hello world!")
matches = matcher(doc)
for match_id, start, end in matches:
    string_id = nlp.vocab.strings[match_id]  # Get string representation
    span = doc[start:end]  # The matched span
    print(match_id, string_id, start, end, span.text)

15578876784678163569 HelloWorld 0 3 Hello, world
15578876784678163569 HelloWorld 4 6 Hello world


In [55]:
import spacy
import re

nlp = spacy.load("en_core_web_sm")
doc = nlp("The United States of America (USA) are commonly known as the United States (U.S. or US) or America.")

expression = r"[Uu](nited|\.?) ?[Ss](tates|\.?)"
for match in re.finditer(expression, doc.text):
    start, end = match.span()
    span = doc.char_span(start, end)
    # This is a Span object or None if match doesn't map to valid token sequence
    if span is not None:
        print("Found match:", span.text)

Found match: United States
Found match: United States
Found match: U.S.
Found match: US


In [69]:
import spacy
import re

nlp = spacy.load("en_core_web_sm")
doc = nlp("The United States of America (USA) are commonly known as the United States (U.S. or US) or America.")

expression = r"[Uu](nited|\.?) ?[Ss](tates|\.?)"
for match in re.finditer(expression, doc.text):
    start, end = match.span()
    span = doc.char_span(start, end)
    # This is a Span object or None if match doesn't map to valid token sequence
    if span is not None:
        print("Found match:", span.text)

Found match: United States
Found match: United States
Found match: U.S.
Found match: US


In [70]:
import spacy
from spacy.matcher import Matcher

nlp = spacy.load("zh_core_web_sm")

doc = nlp("荣耀发布锐龙笔记本，显然会配备7nm工艺、Zen2 架构的"
          "全新锐龙4000 系列，但具体采用低功耗的锐龙4000U 系列，还是高性能的锐龙4000H 系列"
         )

print('-'*50)
for i, token in enumerate(doc):
    print(i, token.text, token.pos_, token.idx)

# 创建匹配模板
pattern1 = [{"POS": "NOUN"},{"TEXT": "笔记本"}]
pattern2 = [{"TEXT": "锐龙"}, {"TEXT": {"REGEX": "\d+"}}, {"TEXT": {"REGEX": "[a-z0-9A-Z_]+"}}]

# 初始化matcher并加入模板
matcher = Matcher(nlp.vocab)
matcher.add("PATTERN1", [pattern1])
matcher.add("PATTERN2", [pattern2])

# 遍历匹配结果
print('-'*50)
for match_id, start, end in matcher(doc):
    # 打印匹配到的字符串名字及匹配到的span的文本
    print(doc.vocab.strings[match_id], doc[start:end].text)

--------------------------------------------------
0 荣耀 NOUN 0
1 发布 VERB 2
2 锐龙 NOUN 4
3 笔记本 NOUN 6
4 ， PUNCT 9
5 显然 ADV 10
6 会 VERB 12
7 配备 VERB 13
8 7nm NUM 15
9 工艺 NOUN 18
10 、 PUNCT 20
11 Zen NOUN 21
12 2 NUM 24
13 架构 NOUN 26
14 的 PART 28
15 全新 ADJ 29
16 锐龙 NOUN 31
17 4000 NUM 33
18 系列 NUM 38
19 ， PUNCT 40
20 但 ADV 41
21 具体 ADV 42
22 采用 VERB 44
23 低 ADJ 46
24 功耗 NOUN 47
25 的 PART 49
26 锐龙 NOUN 50
27 4000 NUM 52
28 U NOUN 56
29 系列 NOUN 58
30 ， PUNCT 60
31 还是 CCONJ 61
32 高性能 NOUN 63
33 的 PART 66
34 锐龙 NOUN 67
35 4000 NUM 69
36 H NUM 73
37 系列 NUM 75
--------------------------------------------------
PATTERN1 锐龙笔记本
PATTERN2 锐龙4000U
PATTERN2 锐龙4000H


In [71]:
import spacy
from spacy.matcher import Matcher

nlp = spacy.load("en_core_web_sm")
matcher = Matcher(nlp.vocab)
pattern = [{"ORTH": "("}, {"SHAPE": "ddd"}, {"ORTH": ")"}, {"SHAPE": "ddd"},
           {"ORTH": "-", "OP": "?"}, {"SHAPE": "ddd"}]
matcher.add("PHONE_NUMBER", [pattern])

doc = nlp("Call me at (123) 456 789 or (123) 456 789!")
print([t.text for t in doc])
matches = matcher(doc)
for match_id, start, end in matches:
    span = doc[start:end]
    print(span.text)

['Call', 'me', 'at', '(', '123', ')', '456', '789', 'or', '(', '123', ')', '456', '789', '!']
(123) 456 789
(123) 456 789


In [109]:
from spacy.lang.en import English
from spacy.matcher import PhraseMatcher

nlp = English()
matcher = PhraseMatcher(nlp.vocab, attr="LOWER")
patterns = [nlp.make_doc(name) for name in ["Angela Merkel", "Barack Obama"]]
matcher.add("Names", patterns)

doc = nlp("angela merkel and us president barack Obama")
for match_id, start, end in matcher(doc):
    print("Matched based on lowercase token text:", doc[start:end])

Matched based on lowercase token text: angela merkel
Matched based on lowercase token text: barack Obama


In [81]:
from spacy.lang.en import English
from spacy.matcher import PhraseMatcher

nlp = English()
matcher = PhraseMatcher(nlp.vocab, attr="SHAPE")
matcher.add("IP", [nlp("127.0.0.1"), nlp("127.127.0.0")])

doc = nlp("Often the router will have an IP address such as 192.168.1.1 or 192.168.2.1.")
for match_id, start, end in matcher(doc):
    print("Matched based on token shape:", doc[start:end])

Matched based on token shape: 192.168.1.1
Matched based on token shape: 192.168.2.1


In [113]:
nlp = spacy.load("zh_core_web_sm")
doc = nlp("荣耀发布锐龙笔记本，显然会配备7nm工艺、Zen2 架构的"
          "全新锐龙4000 系列，但具体采用低功耗的锐龙4000U 系列，还是高性能的锐龙4000H 系列"
         )

for token in doc: 
    print(token.text,  token.pos_, token.head) 

spacy.displacy.render(doc, style='dep') 

荣耀 NOUN 发布
发布 VERB 发布
锐龙 NOUN 笔记本
笔记本 NOUN 发布
， PUNCT 配备
显然 ADV 配备
会 VERB 配备
配备 VERB 发布
7nm NUM 工艺
工艺 NOUN 配备
、 PUNCT Zen
Zen NOUN 2
2 NUM 架构
架构 NOUN 锐龙
的 PART 架构
全新 ADJ 锐龙
锐龙 NOUN 4000
4000 NUM 配备
系列 NUM 4000
， PUNCT 发布
但 ADV 采用
具体 ADV 采用
采用 VERB 发布
低 ADJ 功耗
功耗 NOUN 锐龙
的 PART 功耗
锐龙 NOUN U
4000 NUM U
U NOUN 系列
系列 NOUN 采用
， PUNCT 发布
还是 CCONJ H
高性能 NOUN 锐龙
的 PART 高性能
锐龙 NOUN H
4000 NUM H
H NUM 发布
系列 NUM 发布


In [104]:
span = doc[3:6]
span.root.head 

us

In [114]:
doc.cats

{}

In [116]:
print(nlp.pipe_names)
print(nlp.pipeline)

['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'ner']
[('tok2vec', <spacy.pipeline.tok2vec.Tok2Vec object at 0x7fa45914ffc0>), ('tagger', <spacy.pipeline.tagger.Tagger object at 0x7fa45918eeb8>), ('parser', <spacy.pipeline.dep_parser.DependencyParser object at 0x7fa46c5e0ce0>), ('attribute_ruler', <spacy.pipeline.attributeruler.AttributeRuler object at 0x7fa456c6d308>), ('ner', <spacy.pipeline.ner.EntityRecognizer object at 0x7fa46c5e0a70>)]


In [126]:
import spacy
from spacy.language import Language

# 定义定制化组件
@Language.component(name='length_component')
def length_component(doc):
    # 获取doc的长度
    doc_length = len(doc)
    print(f"This document is {doc_length} tokens long.")
    # 返回这个doc
    return doc


# 读取小规模的中文模型
nlp = spacy.load("zh_core_web_sm")

# 将组件加入到流程的最前面，打印流程组件名
nlp.add_pipe('length_component', first=True)
print(nlp.pipe_names)

# 处理一段文本
doc = nlp("这是一个句子。")

['length_component', 'tok2vec', 'tagger', 'parser', 'attribute_ruler', 'ner']
This document is 4 tokens long.


In [131]:
import spacy
from spacy.matcher import PhraseMatcher
from spacy.tokens import Span
from spacy.language import Language

nlp = spacy.load("zh_core_web_sm")
animals = ["金毛犬", "猫", "乌龟", "老鼠"]
animal_patterns = list(nlp.pipe(animals))
print("animal_patterns:", animal_patterns)
matcher = PhraseMatcher(nlp.vocab)
matcher.add("ANIMAL", animal_patterns)

# 定义定制化组件
@Language.component(name='animal_component')
def animal_component(doc):
    # 把matcher应用到doc上
    matches = matcher(doc)
    # 为每一个匹配结果生成一个Span并赋予标签"ANIMAL"
    spans = [Span(doc, start, end, label='ANIMAL') for match_id, start, end in matches]
    # 用匹配到的span覆盖doc.ents
    doc.ents = spans
    return doc


# 把组件加入到流程中，紧跟在"ner"组件后面
nlp.add_pipe('animal_component', after='ner')
print(nlp.pipe_names)

# 处理文本，打印doc.ents的文本和标签
doc = nlp("我养了一只猫和一条金毛犬。")
import spacyprint([(ent.text, ent.label) for ent in doc.ents])

animal_patterns: [金毛犬, 猫, 乌龟, 老鼠]
['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'ner', 'animal_component']
[('猫', 6303828839600189595), ('金毛犬', 6303828839600189595)]


In [3]:
import spacy 
from spacy.tokens import Token

# 定义取值器函数
def get_is_color(token):
    colors = ["红色", "黄色", "蓝色"]
    return token.text in colors

nlp = spacy.load("zh_core_web_sm")

# 为词符设置有取值器的扩展
Token.set_extension("is_color", getter=get_is_color)

doc = nlp("天空是蓝色的。")
print(doc[2]._.is_color, "-", doc[2].text)

True - 蓝色


In [2]:
import spacy 
from spacy.tokens import Token

# 为Token设置一个有默认值的扩展
Token.set_extension("is_weather", default=False)

nlp = spacy.load("zh_core_web_sm")

doc = nlp("今天回下雨。")

print(doc[2]._.is_weather )
# 覆盖默认扩展特性的值
doc[2]._.is_weather = True
print(doc[2]._.is_weather )

False
True


In [4]:
from spacy.tokens import Span

# 定义取值器函数
def get_has_color(span):
    colors = ["红色", "黄色", "蓝色"]
    return any(token.text in colors for token in span)

# 为Span设置一个带有取值器getter的扩展
Span.set_extension("has_color", getter=get_has_color)

doc = nlp("天空是蓝色的")
print(doc[1:4]._.has_color, "-", doc[1:4].text)
print(doc[0:2]._.has_color, "-", doc[0:2].text)

True - 是蓝色的
False - 天空是


In [5]:
from spacy.tokens import Doc

# 定义含有参数的方法
def has_token(doc, token_text):
    in_doc = token_text in [token.text for token in doc]
    return in_doc

# 在doc上设置方法扩展
Doc.set_extension("has_token", method=has_token)

doc = nlp("天空是蓝色的。")
print(doc._.has_token("蓝色"), "- 蓝色")
print(doc._.has_token("云朵"), "- 云朵")

True - 蓝色
False - 云朵


In [6]:
import spacy
from spacy.tokens import Span

nlp = spacy.load("zh_core_web_sm")


def get_wikipedia_url(span):
    # 如果span有其中一个标签则获取其维基百科URL
    if span.label_ in ("PERSON", "ORG", "GPE", "LOCATION"):
        entity_text = span.text.replace(" ", "_")
        return "https://zh.wikipedia.org/w/index.php?search=" + entity_text


# 设置Span的扩展wikipedia_url及其取值器get_wikipedia_url
Span.set_extension("wikipedia_url", getter=get_wikipedia_url)

doc = nlp(
    "出道这么多年，周杰伦已经成为几代年轻人共同的偶像。"
)
for ent in doc.ents:
    # 打印实体的文本和其维基百科URL
    print(ent.text,  ent._.wikipedia_url)

周杰伦 https://zh.wikipedia.org/w/index.php?search=周杰伦


In [10]:
import json
from spacy.lang.zh import Chinese
from spacy.tokens import Span
from spacy.matcher import PhraseMatcher
from spacy.language import Language

COUNTRIES = ['阿富汗', '奥兰群岛', '阿尔巴尼亚', '阿尔及利亚', '美属萨摩亚', '安道尔', 
             '安哥拉', '安圭拉', '安提瓜和巴布达', '阿根廷', '亚美尼亚', '阿鲁巴', 
             '澳大利亚', '奥地利', '阿塞拜疆', '孟加拉', '巴林', '巴哈马', '巴巴多斯', 
             '白俄罗斯', '比利时', '伯利兹', '贝宁', '百慕大', '不丹', '玻利维亚', 
             '波斯尼亚和黑塞哥维那', '博茨瓦纳', '布维岛', '巴西', '文莱', '保加利亚', 
             '布基纳法索', '布隆迪', '柬埔寨', '喀麦隆', '加拿大', '佛得角', '中非', 
             '乍得', '智利', '圣诞岛', '科科斯群岛', '哥伦比亚', '科摩罗', '刚果', 
             '刚果', '库克群岛', '哥斯达黎加', '科特迪瓦', '中国', '克罗地亚', '古巴', 
             '捷克', '塞浦路斯', '丹麦', '吉布提', '多米尼加', '东帝汶', '厄瓜多尔', '埃及', 
             '赤道几内亚', '厄立特里亚', '爱沙尼亚', '埃塞俄比亚', '法罗群岛', '斐济', '法国', 
             '法国大都会', '法属圭亚那', '法属波利尼西亚', '加蓬', '冈比亚', '格鲁吉亚', '德国', 
             '加纳', '直布罗陀', '希腊', '格林纳达', '瓜德罗普岛', '关岛', '危地马拉', '根西岛', 
             '几内亚比绍', '几内亚', '圭亚那', '海地', '洪都拉斯', '匈牙利', '冰岛', '印度', 
             '印度尼西亚', '伊朗', '伊拉克', '爱尔兰', '马恩岛', '以色列', '意大利', '牙买加', 
             '日本', '泽西岛', '约旦', '哈萨克斯坦', '肯尼亚', '基里巴斯', '韩国', '朝鲜', 
             '科威特', '吉尔吉斯斯坦', '老挝', '拉脱维亚', '黎巴嫩', '莱索托', '利比里亚', 
             '利比亚', '列支敦士登', '立陶宛', '卢森堡', '马其顿', '马拉维', '马来西亚', 
             '马达加斯加', '马尔代夫', '马里', '马耳他', '马绍尔群岛', '马提尼克岛', 
             '毛里塔尼亚', '毛里求斯', '马约特', '墨西哥', '密克罗尼西亚', '摩尔多瓦', 
             '摩纳哥', '蒙古', '黑山', '蒙特塞拉特', '摩洛哥', '莫桑比克', '缅甸', 
             '纳米比亚', '瑙鲁', '尼泊尔', '荷兰', '新喀里多尼亚', '新西兰', '尼加拉瓜', 
             '尼日尔', '尼日利亚', '纽埃', '诺福克岛', '挪威', '阿曼', '巴基斯坦', '帕劳', 
             '巴勒斯坦', '巴拿马', '巴布亚新几内亚', '巴拉圭', '秘鲁', '菲律宾', '皮特凯恩群岛', 
             '波兰', '葡萄牙', '波多黎各', '卡塔尔', '留尼汪岛', '罗马尼亚', '卢旺达', '俄罗斯联邦', 
             '圣赫勒拿', '圣基茨和尼维斯', '圣卢西亚', '圣文森特和格林纳丁斯', '萨尔瓦多', '萨摩亚', 
             '圣马力诺', '圣多美和普林西比', '沙特阿拉伯', '塞内加尔', '塞舌尔', '塞拉利昂', '新加坡', 
             '塞尔维亚', '斯洛伐克', '斯洛文尼亚', '所罗门群岛', '索马里', '南非', '西班牙', '斯里兰卡', 
             '苏丹', '苏里南', '斯威士兰', '瑞典', '瑞士', '叙利亚', '塔吉克斯坦', '坦桑尼亚', '泰国', 
             '特立尼达和多巴哥', '东帝汶', '多哥', '托克劳', '汤加', '突尼斯', '土耳其', '土库曼斯坦', 
             '图瓦卢', '乌干达', '乌克兰', '阿拉伯联合酋长国', '英国', '美国', '乌拉圭', '乌兹别克斯坦', 
             '瓦努阿图', '梵蒂冈', '委内瑞拉', '越南', '瓦利斯群岛和富图纳群岛', '西撒哈拉', '也门', 
             '南斯拉夫', '赞比亚', '津巴布韦']

CAPITALS = {'中国': '北京', '蒙古': '乌兰巴托', '朝鲜': '平壤', '韩国': '首尔', '日本': '东京', 
            '菲律宾': '马尼拉', '印度尼西亚': '雅加达', '文莱': '斯里巴加湾市', '新加坡': '新加坡', '泰国': '曼谷', 
            '马来西亚': '吉隆坡', '越南': '河内', '老挝': '万象', '柬埔寨': '金边', '缅甸': '内比都', '不丹': '廷布', 
            '东帝汶': '帝力', '尼泊尔': '加德满都', '印度': '新德里', '孟加拉国': '达卡', '斯里兰卡': '科伦坡', 
            '马尔代夫': '马累', '巴基斯坦': '伊斯兰堡', '阿富汗': '喀布尔', '塔吉克斯坦': '杜尚别', 
            '吉尔吉斯斯坦': '比什凯克', '哈萨克斯坦': '阿斯塔纳', '乌兹别克期坦': '塔什干', '土库曼斯坦': '阿什哈巴德', 
            '伊朗': '德黑兰', '伊拉克': '巴格达', '科威特': '科威特城', '卡塔尔': '多哈', '阿拉伯联合酋长国': '阿布扎比', 
            '巴林': '麦纳麦', '阿曼': '马斯喀特', '也门': '萨那', '沙特阿拉伯': '利雅得', '约旦': '安曼', 
            '巴勒斯坦': '耶路撒冷', '以色列': '耶路撒冷', '叙利亚': '大马士革', '黎巴嫩': '贝鲁特', '塞浦路斯': '尼科西亚', 
            '土耳其': '安卡拉', '阿塞拜疆': '巴库', '格鲁吉亚': '第比利斯', '亚美尼亚': '埃里温', '挪威': '奥斯路', 
            '冰岛': '雷克雅未克', '瑞典': '斯德哥尔摩', '芬兰': '赫尔辛基', '爱沙尼亚': '塔林', '拉脱维亚': '里加', 
            '立陶宛': '维尔纽斯', '白俄罗斯': '明斯克', '俄罗斯': '莫斯科', '乌克兰': '基辅', '摩尔多瓦': '基希讷乌', 
            '波兰': '华沙', '捷克': '布拉格', '斯洛伐克': '布拉提斯拉发', '匈牙利': '布达佩斯', '德国': '柏林', 
            '英国': '伦敦', '爱尔兰': '都柏林', '丹麦': '哥本哈根', '荷兰': '阿姆斯特丹', '摩纳哥': '摩纳哥', 
            '法国': '巴黎', '比利时': '布鲁塞尔', '卢森堡': '卢森堡', '奥地利': '维也纳', '瑞士': '伯尔尼', 
            '列支敦士登': '瓦杜兹', '西班牙': '马德里', '安道尔': '安道尔', '葡萄牙': '里斯本', '意大利': '罗马', 
            '马耳他': '瓦莱塔', '圣马力诺': '圣马力诺', '梵蒂冈': '梵蒂冈城', '斯洛文尼亚': '卢布尔雅那', 
            '克罗地亚': '萨格勒布', '波斯尼亚和黑塞哥维那': '萨拉热窝', '南斯拉夫': '贝尔格莱德', '马其顿': '斯科普里', 
            '阿尔巴尼亚': '地拉那', '罗马尼亚': '布加勒斯特', '保加利亚': '索非亚', '希腊': '雅典', '埃及': '开罗', 
            '苏丹': '喀土穆', '埃塞俄比亚': '亚的斯亚贝巴', '厄立特里亚': '阿斯马拉', '吉布提': '吉布提', 
            '索马里': '摩加迪沙', '利比亚': '的黎波里', '阿尔及利亚': '阿尔及尔', '突尼斯': '突尼斯', 
            '摩洛哥': '拉巴特', '佛得角': '普拉亚', '毛里塔尼亚': '努瓦克肖特', '马里': '巴马科', '塞内加尔': '达喀尔', 
            '冈比亚': '班珠尔', '几内亚比绍-': '比绍', '几内亚': '科纳克里', '塞拉利昂': '弗里敦', '利比里亚': '蒙罗维亚', 
            '科特迪瓦': '亚穆苏克罗', '布基纳法索': '瓦加杜古', '尼日尔': '尼亚美', '乍得': '恩贾梅纳', 
            '尼日利亚': '阿布贾', '加纳': '阿克拉', '多哥': '洛美', '贝宁': '波多诺伏-', '喀麦隆': '雅温得', 
            '加蓬': '利伯维尔', '赤道几内亚': '马拉博', '圣多美和普林西比': '圣多美', '中非': '班吉', 
            '刚果': '布拉柴维尔', '刚果民主共和国': '金沙萨', '乌干达': '坎帕拉', '卢旺达': '基加利', 
            '布隆迪': '布琼布拉', '坦桑尼亚': '多多马', '肯尼亚': '内罗华', '安哥拉': '罗安达', '赞比亚': '卢萨卡', 
            '马拉维': '利隆圭', '莫桑比克': '马普托', '马达加斯加': '塔那那利佛', '科摩罗': '莫罗尼', 
            '塞舌尔': '维多利亚', '毛里求斯': '路易港', '津巴布韦': '哈拉雷', '博茨瓦纳': '哈博罗内', '纳米比亚': 
            '温得和克', '斯威士兰': '姆巴巴内', '莱索托': '马塞卢', '南非': '比勒陀利亚', '澳大利亚': '堪培拉', 
            '巴布亚新几内亚': '莫尔兹比港', '所罗门群岛': '霍尼亚拉', '瓦努阿图': '维拉港-', '新西兰': '惠灵顿', 
            '斐济': '苏瓦', '汤加': '努库阿洛法', '瑙鲁': '亚伦', '基里巴斯': '塔拉瓦', '图瓦卢': '富纳富提', 
            '萨摩亚': '阿皮亚', '密克罗尼西亚联邦': '帕利基尔', '马绍尔群岛': '马朱罗', '帕劳': '科罗尔', 
            '加拿大': '渥太华', '美国': '华盛顿', '墨西哥': '墨西哥城', '危地马拉': '危地马拉', '伯利兹': '贝尔莫潘', 
            '萨尔瓦多': '圣萨尔瓦多', '洪都拉斯': '特古西加尔巴', '尼加拉瓜': '马那瓜', '哥斯达黎加': '圣何塞', 
            '巴拿马': '巴拿马城', '古巴': '哈瓦那', '巴哈马': '拿骚', '海地': '太子港--', '多米尼加共和国': '圣多明各', 
            '牙买加': '金斯敦', '圣基茨和尼维斯': '巴斯特尔', '安提瓜和巴布达': '圣约翰', '多米尼克': '罗索', 
            '圣卢西亚': '卡斯特里', '圣文森特和格林纳丁斯': '金斯敦', '格林纳达': '圣乔治', '巴巴多斯': '布里奇敦', 
            '特立尼达和多巴哥': '西班牙港--', '哥伦比亚': '波哥大', '厄瓜多尔': '基多', '委内瑞拉': '加拉加斯', 
            '圭亚那-': '乔治敦', '苏里南': '帕拉马里博', '秘鲁': '利马', '玻利维亚': '苏克雷', '巴拉圭': '亚松森', 
            '阿根廷': '布宜诺斯艾利斯', '乌拉圭': '蒙得维的亚', '巴西': '巴西利亚', '智利': '圣地亚哥'}

nlp = Chinese()
matcher = PhraseMatcher(nlp.vocab)
# matcher.add("COUNTRY", None, *list(nlp.pipe(COUNTRIES)))
matcher.add("COUNTRY", list(nlp.pipe(COUNTRIES)))

@Language.component(name='countries_component')
def countries_component(doc):
    # 对所有匹配结果创建一个标签为"GPE"的实体Span
    matches = matcher(doc)
    doc.ents = [Span(doc, start, end, label='GPE') for match_id, start, end in matches]
    return doc


# 把这个组件加入到流程中
#nlp.add_pipe(countries_component)
nlp.add_pipe('countries_component')
print(nlp.pipe_names)

# 取值器，在国家首都的字典中寻找span的文本
get_capital = lambda span: CAPITALS.get(span.text)

# 用这个取值器注册Span的扩展属性"capital"
Span.set_extension('capital', getter=get_capital, force=True) 

# 处理文本，打印实体文本、标签和首都属性
doc = nlp("中国将邦族新加坡和马来西亚一起建造高铁。")
print([(ent.text, ent.label_, ent._.capital) for ent in doc.ents])

['countries_component']
[('中国', 'GPE', '北京'), ('新加坡', 'GPE', '新加坡'), ('马来西亚', 'GPE', '吉隆坡')]


In [18]:
import json
import spacy

nlp = spacy.load("zh_core_web_sm")


TEXTS = ['我最喜欢吃性价比高的麦当劳了！', 
         '我以为麦当劳只有预处理的汉堡，现在我才发现他们家还有生的汉堡？？', 
         '为什么各位还在吃麦当劳 :(', '中国的麦当劳有老北京鸡肉卷，这也太爽了！', 
         '作为一个帅帅的男人，去麦当劳我只吃巨无霸:P', 
         '今天早上决定去吃麦当劳套餐，现在胃里涨了一整天了。']

docs = list(nlp.pipe(TEXTS))
# 处理文本，打印形容词
for doc in docs:
    print([token.text for token in doc if token.pos_ == "ADJ"])

[]
[]
[]
['老']
[]
[]


In [21]:
import json
from spacy.lang.en import English
from spacy.tokens import Doc

DATA = [['One morning, when Gregor Samsa woke from troubled dreams, he found himself transformed in his bed into a horrible vermin.', {'author': 'Franz Kafka', 'book': 'Metamorphosis'}], ["I know not all that may be coming, but be it what it will, I'll go to it laughing.", {'author': 'Herman Melville', 'book': 'Moby-Dick or, The Whale'}], ['It was the best of times, it was the worst of times.', {'author': 'Charles Dickens', 'book': 'A Tale of Two Cities'}], ['The only people for me are the mad ones, the ones who are mad to live, mad to talk, mad to be saved, desirous of everything at the same time, the ones who never yawn or say a commonplace thing, but burn, burn, burn like fabulous yellow roman candles exploding like spiders across the stars.', {'author': 'Jack Kerouac', 'book': 'On the Road'}], ['It was a bright cold day in April, and the clocks were striking thirteen.', {'author': 'George Orwell', 'book': '1984'}], ['Nowadays people know the price of everything and the value of nothing.', {'author': 'Oscar Wilde', 'book': 'The Picture Of Dorian Gray'}]]

nlp = English()

# 注册Doc的扩展"author"（默认值为None）
Doc.set_extension('author', default=None, force=True) 

# 注册Doc的扩展"book"（默认值为None）
Doc.set_extension('book', default=None, force=True) 

for doc, context in nlp.pipe(DATA, as_tuples=True):
    # 从context中设置属性doc._.book和doc._.author
    doc._.book = context['book']
    doc._.author = context['author']

    # 打印文本和定制化的属性数据
    print(f"{doc.text}\n — '{doc._.book}' by {doc._.author}\n")

One morning, when Gregor Samsa woke from troubled dreams, he found himself transformed in his bed into a horrible vermin.
 — 'Metamorphosis' by Franz Kafka

I know not all that may be coming, but be it what it will, I'll go to it laughing.
 — 'Moby-Dick or, The Whale' by Herman Melville

It was the best of times, it was the worst of times.
 — 'A Tale of Two Cities' by Charles Dickens

The only people for me are the mad ones, the ones who are mad to live, mad to talk, mad to be saved, desirous of everything at the same time, the ones who never yawn or say a commonplace thing, but burn, burn, burn like fabulous yellow roman candles exploding like spiders across the stars.
 — 'On the Road' by Jack Kerouac

It was a bright cold day in April, and the clocks were striking thirteen.
 — '1984' by George Orwell

Nowadays people know the price of everything and the value of nothing.
 — 'The Picture Of Dorian Gray' by Oscar Wilde



In [22]:
import spacy

nlp = spacy.load("zh_core_web_sm")
text = (
    "在300多年的风雨历程中，历代同仁堂人始终恪守“炮制虽繁必不敢省人工，品味虽贵必不敢减物力”的古训，"
    "树立“修合无人见，存心有天知”的自律意识，造就了制药过程中兢兢小心、精益求精的严细精神。"
)

doc = nlp(text)
print([token.text for token in doc])

# 仅对文本做分词
doc = nlp.make_doc(text)
print([token.text for token in doc])

['在', '300多', '年', '的', '风雨', '历程', '中', '，', '历代', '同仁', '堂人', '始终', '恪守', '“', '炮制', '虽', '繁必', '不', '敢', '省', '人工', '，', '品味', '虽', '贵必', '不', '敢', '减物力', '”', '的', '古训', '，', '树立', '“', '修合', '无', '人', '见', '，', '存心', '有', '天知', '”', '的', '自律', '意识', '，', '造就', '了', '制药', '过程', '中', '兢兢小心', '、', '精益求精', '的', '严细', '精神', '。']
['在', '300多', '年', '的', '风雨', '历程', '中', '，', '历代', '同仁', '堂人', '始终', '恪守', '“', '炮制', '虽', '繁必', '不', '敢', '省', '人工', '，', '品味', '虽', '贵必', '不', '敢', '减物力', '”', '的', '古训', '，', '树立', '“', '修合', '无', '人', '见', '，', '存心', '有', '天知', '”', '的', '自律', '意识', '，', '造就', '了', '制药', '过程', '中', '兢兢小心', '、', '精益求精', '的', '严细', '精神', '。']


In [27]:
import spacy

nlp = spacy.load("zh_core_web_sm")
text = (
    "在300多年的风雨历程中，历代同仁堂人始终恪守“炮制虽繁必不敢省人工，品味虽贵必不敢减物力”的古训，"
    "树立“修合无人见，存心有天知”的自律意识，造就了制药过程中兢兢小心、精益求精的严细精神。"
)

print(nlp.pipe_names)

doc = nlp(text)
print(doc.ents)
    
# 关闭tagger和parser
with nlp.disable_pipes("tagger", "parser", 'ner'):
    # 处理文本
    doc = nlp(text)
    print(doc.ents)

['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'ner']
(300多年,)
()


In [30]:
import spacy

nlp = spacy.load("zh_core_web_sm")
text = (
    "在300多年的风雨历程中，历代同仁堂人始终恪守“炮制虽繁必不敢省人工，品味虽贵必不敢减物力”的古训，"
    "树立“修合无人见，存心有天知”的自律意识，造就了制药过程中兢兢小心、精益求精的严细精神。"
)

doc = nlp(text)
print([(token.text, token.pos_) for token in doc])

# 仅对文本做分词
doc = nlp.make_doc(text)
print([(token.text, token.pos_) for token in doc])


[('在', 'ADP'), ('300多', 'NUM'), ('年', 'NUM'), ('的', 'PART'), ('风雨', 'NOUN'), ('历程', 'NOUN'), ('中', 'PART'), ('，', 'PUNCT'), ('历代', 'VERB'), ('同仁', 'NOUN'), ('堂人', 'NOUN'), ('始终', 'ADV'), ('恪守', 'VERB'), ('“', 'PUNCT'), ('炮制', 'NOUN'), ('虽', 'SCONJ'), ('繁必', 'NUM'), ('不', 'ADV'), ('敢', 'VERB'), ('省', 'VERB'), ('人工', 'VERB'), ('，', 'PUNCT'), ('品味', 'NOUN'), ('虽', 'SCONJ'), ('贵必', 'VERB'), ('不', 'ADV'), ('敢', 'VERB'), ('减物力', 'NOUN'), ('”', 'PUNCT'), ('的', 'PART'), ('古训', 'NOUN'), ('，', 'PUNCT'), ('树立', 'VERB'), ('“', 'PUNCT'), ('修合', 'VERB'), ('无', 'VERB'), ('人', 'NOUN'), ('见', 'VERB'), ('，', 'PUNCT'), ('存心', 'ADV'), ('有', 'VERB'), ('天知', 'NOUN'), ('”', 'PUNCT'), ('的', 'PART'), ('自律', 'NOUN'), ('意识', 'NOUN'), ('，', 'PUNCT'), ('造就', 'VERB'), ('了', 'PART'), ('制药', 'NOUN'), ('过程', 'NOUN'), ('中', 'PART'), ('兢兢小心', 'VERB'), ('、', 'PUNCT'), ('精益求精', 'VERB'), ('的', 'PART'), ('严细', 'NOUN'), ('精神', 'NOUN'), ('。', 'PUNCT')]
[('在', ''), ('300多', ''), ('年', ''), ('的', ''), ('风雨', ''), ('历程', ''), ('

In [51]:
import json
from spacy.matcher import Matcher


TEXTS = ['如何预定 iPhone X', 
         'iPhone X就要来了', 
         '为买一个iPhone X花上万块钱值得吗？', 
         'iPhone 8的评测出来了', 
         'iPhone 11 vs iPhone 8：有哪些升级？', 
         '我急需一部新手机，给点建议吧！']

nlp = spacy.load("zh_core_web_sm")
matcher = Matcher(nlp.vocab)

# 两个词符，其小写形式匹配到"iphone"和"x"上
pattern1 = [{"LOWER": "iphone"}, {"LOWER": "x"}]

# 词符的小写形式匹配到"iphone"和一个数字上
pattern2 = [{"LOWER": "iphone"}, {"IS_DIGIT": True}]

TRAINING_DATA = []

# 把模板加入到matcher中然后检查结果
matcher.add("GADGET", [pattern1, pattern2])
for doc in nlp.pipe(TEXTS):
    # 在doc上做匹配，创建一个匹配结果span的列表
    spans = [doc[start:end] for match_id, start, end in matcher(doc)]
    # 获取(start character, end character, label)这样的匹配结果元组
    entities = [(span.start_char, span.end_char, "GADGET") for span in spans]
    # 将匹配结果的格式变为(doc.text, entities)元组
    training_example = (doc.text, {"entities": entities})
    # 把这些例子加入到训练数据中
    TRAINING_DATA.append(training_example)
    
print(*TRAINING_DATA, sep="\n")

('如何预定 iPhone X', {'entities': [(5, 13, 'GADGET')]})
('iPhone X就要来了', {'entities': [(0, 8, 'GADGET')]})
('为买一个iPhone X花上万块钱值得吗？', {'entities': [(4, 12, 'GADGET')]})
('iPhone 8的评测出来了', {'entities': [(0, 8, 'GADGET')]})
('iPhone 11 vs iPhone 8：有哪些升级？', {'entities': [(0, 9, 'GADGET'), (13, 21, 'GADGET')]})
('我急需一部新手机，给点建议吧！', {'entities': []})


In [ ]:
# 循环重复10次
for i in range(10):
    # 随机化训练数据的顺序
    random.shuffle(TRAINING_DATA)
    # 创建批次并遍历
    for batch in spacy.util.minibatch(TRAINING_DATA):
    # 在该批数据中分离文本和标注
        texts = [text for text, annotation in batch]
        annotations = [annotation for text, annotation in batch]
        # 更新模型
        nlp.update(texts, annotations)

# 保存模型
nlp.to_disk(path_to_model)

In [54]:
# 从空的中文模型开始
nlp = spacy.blank("zh")
# 创建一个空的实体识别器加入到流程中
ner = nlp.create_pipe("ner")
nlp.add_pipe('ner')
# 添加一个新的标签
ner.add_label("GADGET")

# 开始训练
nlp.begin_training()
# 训练10个循环
for itn in range(10):
    random.shuffle(examples)
    # 将例子切分为一系列批次
    for batch in spacy.util.minibatch(examples, size=2):
        texts = [text for text, annotation in batch]
        annotations = [annotation for text, annotation in batch]
        # 更新模型
        nlp.update(texts, annotations)

NameError: name 'examples' is not defined

In [82]:
import spacy
import random
import json
from spacy.training import Example
    
TRAINING_DATA = [['如何预定iPhone X', {'entities': [[4, 12, 'GADGET']]}], 
                 ['iPhone X就要来了', {'entities': [[0, 8, 'GADGET']]}], 
                 ['为买一个iPhone X花上万块钱值得吗？', {'entities': [[4, 12, 'GADGET']]}], 
                 ['iPhone 8的评测出来了', {'entities': [[0, 8, 'GADGET']]}], 
                 ['最新的iPhone已经到第11代了', {'entities': [[3, 9, 'GADGET']]}], 
                 ['我急需一部新手机，给点建议吧！', {'entities': []}]]

nlp = spacy.blank("zh")
ner = nlp.create_pipe("ner")
nlp.add_pipe("ner")
ner.add_label("GADGET")

examples = []
for text, annots in TRAINING_DATA:    
    examples.append(Example.from_dict(nlp.make_doc(text), annots))
    
nlp.initialize(lambda: examples)

# 开始训练
nlp.begin_training()

# 迭代10个循环
epochs = 10  
for itn in range(epochs):
    losses = {}
    random.shuffle(examples)
    for batch in spacy.util.minibatch(examples, size=8):
        nlp.update(batch, losses=losses) 
    print(f'{itn+1}/{epochs}', losses)

1/10 {'ner': 42.49999928474426}
2/10 {'ner': 68.98857140541077}
3/10 {'ner': 71.35534292459488}
4/10 {'ner': 68.32567793130875}
5/10 {'ner': 63.826959669589996}
6/10 {'ner': 57.02356398105621}
7/10 {'ner': 47.38814955949783}
8/10 {'ner': 33.9421991109848}
9/10 {'ner': 21.177328035235405}
10/10 {'ner': 11.303111843764782}


In [84]:
doc = nlp('最新的iPhone已经到第11代了, 你好，中国 ')
print(doc.ents)
for ent in doc.ents:
    print(ent.text, ent.label_)

()


In [104]:
TRAINING_DATA = [
    (
        "哔哩哔哩与阿里巴巴合作为博主们建立社群",
        {"entities": [(0, 4, "WEBSITE"), (5, 9, "WEBSITE")]},
    ),
    ("李子柒打破了Youtube的记录", {"entities": [(6, 13, "WEBSITE")]}),
    (
        "阿里巴巴的创始人马云提供了一千万购物优惠券",
        {"entities": [(0, 4, "WEBSITE")]},
    ),
    # 以及更多类似的数据。。。
]

nlp = spacy.load("zh_core_web_sm")

texts = [data[0] for data in TRAINING_DATA]
for doc in nlp.pipe(texts):
    print([(token.text, token.idx) for token in doc])

for text, entities in TRAINING_DATA:
    print('-'*50) 
    doc = nlp(text) 
    for start, end, label in entities["entities"]:
        print(start, end, label) 
        print(doc.char_span(start, end, label=label))

[('哔哩', 0), ('哔哩', 2), ('与', 4), ('阿里巴巴', 5), ('合作', 9), ('为', 11), ('博主们', 12), ('建立', 15), ('社群', 17)]
[('李子柒', 0), ('打破', 3), ('了', 5), ('Youtube', 6), ('的', 13), ('记录', 14)]
[('阿里巴巴', 0), ('的', 4), ('创始人', 5), ('马云', 8), ('提供', 10), ('了', 12), ('一千万', 13), ('购物', 16), ('优惠券', 18)]
--------------------------------------------------
0 4 WEBSITE
哔哩哔哩
5 9 WEBSITE
阿里巴巴
--------------------------------------------------
6 13 WEBSITE
Youtube
--------------------------------------------------
0 4 WEBSITE
阿里巴巴


In [105]:
import pandas as pd
from tqdm import tqdm
import spacy
from spacy.tokens import DocBin

def get_train_data():
    TEXTS = ['如何预定 iPhone X', 
             'iPhone X就要来了', 
             '为买一个iPhone X花上万块钱值得吗？', 
             'iPhone 8的评测出来了', 
             'iPhone 11 vs iPhone 8：有哪些升级？', 
             '我急需一部新手机，给点建议吧！']

    nlp = spacy.load("zh_core_web_sm")
    matcher = Matcher(nlp.vocab)

    # 两个词符，其小写形式匹配到"iphone"和"x"上
    pattern1 = [{"LOWER": "iphone"}, {"LOWER": "x"}]

    # 词符的小写形式匹配到"iphone"和一个数字上
    pattern2 = [{"LOWER": "iphone"}, {"IS_DIGIT": True}]

    TRAIN_DATA = []

    # 把模板加入到matcher中然后检查结果
    matcher.add("GADGET", [pattern1, pattern2])
    for doc in nlp.pipe(TEXTS):
        # 在doc上做匹配，创建一个匹配结果span的列表
        spans = [doc[start:end] for match_id, start, end in matcher(doc)]
        # 获取(start character, end character, label)这样的匹配结果元组
        entities = [(span.start_char, span.end_char, "GADGET") for span in spans]
        # 将匹配结果的格式变为(doc.text, entities)元组
        training_example = (doc.text, {"entities": entities})
        # 把这些例子加入到训练数据中
        TRAIN_DATA.append(training_example)

    print(*TRAIN_DATA, sep="\n")    
    return TRAIN_DATA

TRAIN_DATA = get_train_data()


('如何预定 iPhone X', {'entities': [(5, 13, 'GADGET')]})
('iPhone X就要来了', {'entities': [(0, 8, 'GADGET')]})
('为买一个iPhone X花上万块钱值得吗？', {'entities': [(4, 12, 'GADGET')]})
('iPhone 8的评测出来了', {'entities': [(0, 8, 'GADGET')]})
('iPhone 11 vs iPhone 8：有哪些升级？', {'entities': [(0, 9, 'GADGET'), (13, 21, 'GADGET')]})
('我急需一部新手机，给点建议吧！', {'entities': []})


In [107]:

nlp = spacy.blank("en") # load a new spacy model
db = DocBin() # create a DocBin object

for text, annot in tqdm(TRAIN_DATA): # data in previous format
    doc = nlp.make_doc(text) # create doc object from text
    ents = []
    for start, end, label in annot["entities"]: # add character indexes
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is not None:
            ents.append(span)
    doc.ents = ents # label the text with the ents
    db.add(doc)

db.to_disk("./train.spacy") # save the docbin object

100%|██████████| 6/6 [00:00<00:00, 1300.42it/s]


In [113]:
import numpy as np
np.random.choice(a=5, size=3, replace=True, p=[0.2, 0.1, 0.3, 0.4, 0.0])

array([2, 3, 3])